In [1]:
import dao.MongoUsersSong as mus
import panda.SongUser as supd
# 展示操作一个用户的时候应该如何使用
mongomus = mus.UserSongRecom(user_id="1a78b764b67911ed95dd00155dadb10b")
# mongomus.makeRecomUsersSetAnswer(limit="ONE")

In [2]:
# 获取recom中之前设置的数据
projections = {"_id": 0, "id": 1, "name": 1, "RecomUsers": 1}
doc = mongomus.rdb.findDocument(
    collection_name="recom", query=mongomus.query, projection=projections)
print(doc)
# 储存请求用户的名字
mongomus.user_name = doc[0]['name']
# 跟据数据库给的RecUsers来处理数据
list_users_id = []
list_users_id.append(mongomus.user_id)
# print(self.user_id, list_users_id)
# users_id 合集，之后会通过这个来形成x轴坐标
list_users_id.extend(doc[0]['RecomUsers'])
# 构成xy轴的内容源头
docs = mongomus.getSongsWithUserIdFromMongo(array=list_users_id)
# print(docs)

[{'id': '1a78b764b67911ed95dd00155dadb10b', 'name': 'cLCH', 'RecomUsers': ['1a78c984b67911ed95dd00155dadb10b', '1a78de4cb67911ed95dd00155dadb10b', '1a7922eeb67911ed95dd00155dadb10b', '1a7904e4b67911ed95dd00155dadb10b', '1a78b0c0b67911ed95dd00155dadb10b', '1a78ae7cb67911ed95dd00155dadb10b', '1a78e36ab67911ed95dd00155dadb10b', '1a791bf0b67911ed95dd00155dadb10b', '1a78c22cb67911ed95dd00155dadb10b', '1a78c6a0b67911ed95dd00155dadb10b', '1a78bc78b67911ed95dd00155dadb10b', '1a78b962b67911ed95dd00155dadb10b', '1a78e414b67911ed95dd00155dadb10b', '1a78cd12b67911ed95dd00155dadb10b', '1a791aceb67911ed95dd00155dadb10b', '1a78b034b67911ed95dd00155dadb10b', '1a78c74ab67911ed95dd00155dadb10b', '1a78ba0cb67911ed95dd00155dadb10b', '1a78ba8eb67911ed95dd00155dadb10b', '1a78e1eeb67911ed95dd00155dadb10b', '1a78d0fab67911ed95dd00155dadb10b', '1a78e7b6b67911ed95dd00155dadb10b', '1a78d3e8b67911ed95dd00155dadb10b', '1a78b3d6b67911ed95dd00155dadb10b', '1a79064cb67911ed95dd00155dadb10b', '1a78b1f6b67911ed95dd0015

In [3]:
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data = mongomus.makeXYMatric(docs=docs)
# print(data)
sudf = supd.SUPandas(datas=list(
    map(list, data.toarray())), items=mongomus.y, users=mongomus.x)
    

In [4]:
sudf.df

,cLCH,WfTR2,zFpXHZ0n,mdOQjX,m3NOW7Pp,G,Z,c4D,9D0ZA,UnfVc1Tg,...,dyrvSYZxx,1C3ijya,KuF,D0nHKfvH,ud3K,FnJi,GcTaF8d,afqmIx79w,m5eKRJ,zAaptCORtu
491529,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
567083018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29097995,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29196302,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28377108,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3571692,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1540081,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400162805,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1540090,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# 推荐过程
# dict_user_song = mongomus.makeRecomDcition(df_object=sudf, sign=-1)
similar = sudf.makeSimilarityBetweenUser()

/home/arki/.local/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [6]:
similar

,cLCH,WfTR2,zFpXHZ0n,mdOQjX,m3NOW7Pp,G,Z,c4D,9D0ZA,UnfVc1Tg,...,dyrvSYZxx,1C3ijya,KuF,D0nHKfvH,ud3K,FnJi,GcTaF8d,afqmIx79w,m5eKRJ,zAaptCORtu
cLCH,1.000000,0.016920,0.021576,0.015598,0.017171,0.014774,0.017500,0.015343,0.018456,0.015460,...,0.013289,0.015351,0.007389,0.004043,0.009744,0.019005,0.011673,0.008653,0.001441,0.020426
WfTR2,0.016920,1.000000,0.015195,0.014085,0.027708,0.016008,0.011794,0.019373,0.015306,0.017399,...,0.009019,0.012291,0.005038,0.004155,0.011250,0.015611,0.014925,0.006321,0.002972,0.017256
zFpXHZ0n,0.021576,0.015195,1.000000,0.022638,0.016468,0.010460,0.008341,0.016393,0.019811,0.009964,...,0.012953,0.008906,0.007353,0.000000,0.008696,0.018462,0.019101,0.010355,0.001783,0.022052
mdOQjX,0.015598,0.014085,0.022638,1.000000,0.011864,0.016473,0.013853,0.012241,0.013055,0.022184,...,0.008159,0.008055,0.007874,0.004335,0.014323,0.011278,0.017472,0.007895,0.006240,0.017715
m3NOW7Pp,0.017171,0.027708,0.016468,0.011864,1.000000,0.013562,0.015523,0.015044,0.015612,0.023368,...,0.010787,0.008511,0.005988,0.003927,0.002356,0.013239,0.013359,0.007212,0.001397,0.017500
G,0.014774,0.016008,0.010460,0.016473,0.013562,1.000000,0.016667,0.014170,0.015829,0.015343,...,0.008917,0.015171,0.004335,0.006472,0.008571,0.016211,0.009912,0.013158,0.003509,0.012230
Z,0.017500,0.011794,0.008341,0.013853,0.015523,0.016667,1.000000,0.014440,0.012521,0.015472,...,0.006608,0.010929,0.008642,0.004049,0.004854,0.010782,0.007759,0.007417,0.004342,0.016115
c4D,0.015343,0.019373,0.016393,0.012241,0.015044,0.014170,0.014440,1.000000,0.008123,0.023111,...,0.025063,0.008516,0.009804,0.001546,0.012448,0.011799,0.007479,0.009831,0.003356,0.011009
9D0ZA,0.018456,0.015306,0.019811,0.013055,0.015612,0.015829,0.012521,0.008123,1.000000,0.019737,...,0.012277,0.011013,0.009975,0.004087,0.006127,0.016364,0.019763,0.006227,0.007331,0.018820
UnfVc1Tg,0.015460,0.017399,0.009964,0.022184,0.023368,0.015343,0.015472,0.023111,0.019737,1.000000,...,0.015102,0.007407,0.009569,0.006527,0.005882,0.015873,0.011385,0.010804,0.000000,0.021685


In [7]:
# 使用平均数，对以上的相似度结果求平均，并取高于平均数的对象形成一个list
diction_user = sudf.makeTopNUsers(similar=similar,sign=1)
diction = sudf.makeRecomUserBySong(
            topN=diction_user, song_num=50)

In [8]:
# 程序过程中会有sign 来判断是去单个，还是返回所有的
dict_result = {mongomus.user_name: diction[mongomus.user_name]}

In [9]:
print(dict_result)

{'cLCH': [35504129, 29769734, 491529, 29097995, 1309261838, 28377108, 516358165, 29999126, 40255512, 1456603168, 4472867, 25657383, 26116136, 502218792, 29999146, 524331, 1830469676, 1605671, 622645, 426852405, 23986232, 41631802, 31260730, 1333346369, 404783172, 27836485, 28953086, 425295943, 2195529, 479281226, 570556489, 545407050, 545407049, 1785940, 1495384148, 3571798, 1355169880, 1433108569, 344152, 1411121242, 28967003, 1303019613, 21184600, 471613535, 1490124896, 1490124895, 501514336, 1949795, 16466019, 4128869]}


In [10]:
# 最后保存
# mongomus.saveUserSongRecomAnswer(diction=dict_result)
docs = mongomus.changeDataFormat(diction=dict_result, list_keys=list(dict_result))
# print(docs)
mongomus.wdb.writeDocument(
    docs=docs, collection_name="recom")


update coll cLCH in recom
